In [1]:
# Collection
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

df_all = pd.DataFrame()
rows = []
for year in range(2000, 2024):
  if year % 20 == 10:
      time.sleep(61)
  dir = "https://www.basketball-reference.com/leagues/NBA_" + str(year) + "_per_game.html"
  df_current = pd.read_html(dir)
  df_current[0]["Year"] = year
  df_all = pd.concat([df_all, df_current[0]])

In [2]:
df_all

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Tariq Abdul-Wahad,SG,25,TOT,61,56,25.9,4.5,10.6,...,1.7,3.1,4.8,1.6,1.0,0.5,1.7,2.4,11.4,2000
1,1,Tariq Abdul-Wahad,SG,25,ORL,46,46,26.2,4.8,11.2,...,1.7,3.5,5.2,1.6,1.2,0.3,1.9,2.5,12.2,2000
2,1,Tariq Abdul-Wahad,SG,25,DEN,15,10,24.9,3.4,8.7,...,1.6,1.9,3.5,1.7,0.4,0.8,1.3,2.1,8.9,2000
3,2,Shareef Abdur-Rahim,SF,23,VAN,82,82,39.3,7.2,15.6,...,2.7,7.4,10.1,3.3,1.1,1.1,3.0,3.0,20.3,2000
4,3,Cory Alexander,PG,26,DEN,29,2,11.3,1.0,3.4,...,0.3,1.2,1.4,2.0,0.8,0.1,1.0,1.3,2.8,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,535,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,...,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2023
701,536,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,...,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2023
702,537,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,...,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2023
703,538,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,...,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,2023


In [3]:
# Cleaning
df_all = df_all[df_all['Player'] != 'Player']
print(df_all.dtypes)
columns_to_convert = [
    'Rk','Age','G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA',
    '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
    'BLK', 'TOV', 'PF', 'PTS', 'Year'
]
df_all[columns_to_convert] = df_all[columns_to_convert].astype(float)
print(df_all.dtypes)

duplicates_to_remove = df_all.duplicated(subset=['Player', 'Year']) & ~((df_all['Tm'] == 'TOT') & (df_all.duplicated(subset=['Player', 'Year', 'Tm'])))
df_no_duplicates = df_all[~duplicates_to_remove]
df_no_duplicates
min_games = df_no_duplicates[df_no_duplicates["G"].astype(int) >= 30]

def position_to_position(c):
  if c == "SF-SG":
    return "SF"
  elif c == "PG-SG":
    return "PG"
  elif c == "PF-SF":
    return "PF"
  elif c == "C-PF":
    return "C"
  elif c == "SG-SF":
    return "SG"
  elif c == "SG-PG":
    return "SG"
  elif c == "SF-PF":
    return "SF"
  elif c == "PF-C":
    return "PF"
  elif c == "SG-PF":
    return "SG"
  elif c == "PG-SF":
    return "PG"
  elif c == "SG-PG-SF":
    return "SG"
  else:
    return c
def position_to_3(c):
  if c == "PG":
    return "G"
  elif c == "SG":
    return "G"
  elif c == "SF":
    return "F"
  elif c == "PF":
    return "F"
  elif c == "C":
    return "C"
min_games["Pos_3"] = min_games["Pos"].map(position_to_3)
min_games["Pos"] = min_games["Pos"].map(position_to_position)
min_games.to_csv("nba_21st_cent.csv")
min_games


Rk        object
Player    object
Pos       object
Age       object
Tm        object
G         object
GS        object
MP        object
FG        object
FGA       object
FG%       object
3P        object
3PA       object
3P%       object
2P        object
2PA       object
2P%       object
eFG%      object
FT        object
FTA       object
FT%       object
ORB       object
DRB       object
TRB       object
AST       object
STL       object
BLK       object
TOV       object
PF        object
PTS       object
Year       int64
dtype: object


<ipython-input-3-cc96301f3682>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all[columns_to_convert] = df_all[columns_to_convert].astype(float)
<ipython-input-3-cc96301f3682>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  min_games["Pos_3"] = min_games["Pos"].map(position_to_3)
<ipython-input-3-cc96301f3682>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

Rk        float64
Player     object
Pos        object
Age       float64
Tm         object
G         float64
GS        float64
MP        float64
FG        float64
FGA       float64
FG%       float64
3P        float64
3PA       float64
3P%       float64
2P        float64
2PA       float64
2P%       float64
eFG%      float64
FT        float64
FTA       float64
FT%       float64
ORB       float64
DRB       float64
TRB       float64
AST       float64
STL       float64
BLK       float64
TOV       float64
PF        float64
PTS       float64
Year      float64
dtype: object


,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Pos_3
0,1.0,Tariq Abdul-Wahad,SG,25.0,TOT,61.0,56.0,25.9,4.5,10.6,...,3.1,4.8,1.6,1.0,0.5,1.7,2.4,11.4,2000.0,G
3,2.0,Shareef Abdur-Rahim,SF,23.0,VAN,82.0,82.0,39.3,7.2,15.6,...,7.4,10.1,3.3,1.1,1.1,3.0,3.0,20.3,2000.0,F
5,4.0,Ray Allen*,SG,24.0,MIL,82.0,82.0,37.4,7.8,17.2,...,3.4,4.4,3.8,1.3,0.2,2.2,2.3,22.1,2000.0,G
7,6.0,John Amaechi,C,29.0,ORL,80.0,53.0,21.1,3.8,8.8,...,2.6,3.3,1.2,0.4,0.5,1.7,2.0,10.5,2000.0,C
8,7.0,Derek Anderson,SG,25.0,LAC,64.0,58.0,34.4,5.9,13.4,...,2.8,4.0,3.4,1.4,0.2,2.6,2.3,16.9,2000.0,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,531.0,Christian Wood,C,27.0,DAL,67.0,17.0,25.9,5.9,11.5,...,6.0,7.3,1.8,0.4,1.1,1.8,2.5,16.6,2023.0,C
697,532.0,Delon Wright,PG,30.0,WAS,50.0,14.0,24.4,2.8,5.8,...,2.4,3.6,3.9,1.8,0.3,0.9,1.2,7.4,2023.0,G
700,535.0,Thaddeus Young,PF,34.0,TOR,54.0,9.0,14.7,2.0,3.7,...,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2023.0,F
701,536.0,Trae Young,PG,24.0,ATL,73.0,73.0,34.8,8.2,19.0,...,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2023.0,G
